![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx and BigCode `starcoder-15.5b` to generate code based on instruction

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support for code generating in Watsonx. It introduces commands for defining prompt and model testing.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to generate code using BigCode `starcoder-15.5b` Watsonx model based on instruction provided by the user.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on Watsonx](#models)
- [Generate code based on instruction](#generate)
- [Generated code testing](#testing)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Install dependecies

In [ ]:
!pip install "ibm-watson-machine-learning>=1.0.321" | tail -n 1

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [ ]:
wml_credentials = {
    "username": username,
    "apikey": api_key,
    "url": url,
    "instance_id": 'openshift',
    "version": '4.8'
}

Alternatively you can use `username` and `password` to authenticate WML services.

```
wml_credentials = {
    "username": ***,
    "password": ***,
    "url": ***,
    "instance_id": 'openshift',
    "version": '4.8'
}

```

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [2]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All avaliable models are presented under ModelTypes class.
For more information refer to [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#ibm_watson_machine_learning.foundation_models.utils.enums.ModelTypes).

In [3]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT']


You need to specify `model_id` that will be used for inferencing:

In [4]:
model_id = ModelTypes.STARCODER

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#metanames.GenTextParamsMetaNames).

In [13]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.STOP_SEQUENCES: ["<end·of·code>"]
}

### Initialize the model
Initialize the `Model` class with previous set params.

In [14]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id=model_id, 
    params=parameters, 
    credentials=wml_credentials,
    project_id=project_id)

### Model's details

In [15]:
model.get_details()

{'model_id': 'bigcode/starcoder',
 'label': 'starcoder-15.5b',
 'provider': 'BigCode',
 'source': 'Hugging Face',
 'short_description': 'The StarCoder models are 15.5B parameter models that can generate code from natural language descriptions',
 'long_description': 'The StarCoder models are 15.5B parameter models trained on 80+ programming languages from The Stack (v1.2), with opt-out requests excluded. The model uses Multi Query Attention, a context window of 8192 tokens, and was trained using the Fill-in-the-Middle objective on 1 trillion tokens.',
 'task_ids': ['code'],
 'tasks': [{'id': 'retrieval_augmented_generation',
   'ratings': {'quality': 1}}],
 'model_limits': {'max_sequence_length': 8192},
 'limits': {'lite': {'max_output_tokens': 1024},
  'v2-professional': {'max_output_tokens': 1024},
  'v2-standard': {'max_output_tokens': 1024}},
 'min_shot_size': 0,
 'tier': 'class_2',
 'number_params': '15.5b'}

<a id="generate"></a>
## Generate code based on instruction

Define instructions for the model with at-least one example.

In [16]:
instruction = """Using the directions below, generate Python code for the given task.

Input:
# Write a Python function that prints 'Hello World!' string 'n' times.

Output:
def print_n_times(n):
    for i in range(n):
        print("Hello World!")

<end of code>
"""

Prepare question for the model.

In [17]:
question = """Input:
# Write a Python function, which generates sequence of prime numbers.
# The function 'primes' will take the argument 'n', an int. It will return a list which contains all primes less than 'n'."""

### Generat the code using BigCode `starcoder-15.5b` model.


Inter the model to generate the code, according to provided instruction.

In [18]:
result = model.generate_text(" ".join([instruction, question]))

Formatting the text to get the function itself

In [19]:
code_as_text = result.split('Output:')[1].split('<end of code>')[0]

<a id="testing"></a>
## Generated code testing

The resulting code looks as below.

In [20]:
print(code_as_text)


def primes(n):
    primes = []
    for i in range(2, n):
        for j in range(2, i):
            if i % j == 0:
                break
        else:
            primes.append(i)
    return primes




Use generated code to make it as function.

In [21]:
exec(code_as_text)

Define the number 'n' for which the primes() function should process prime numbers.

In [22]:
n = 25

Test and run the generated function.

In [23]:
primes(n)

[2, 3, 5, 7, 11, 13, 17, 19, 23]

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to generate code based on instuction with BigCode `starcoder-15.5b` on Watsonx. 
 
 Check out our _[Online Documentation]()_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors: 

 **Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2023-2025 IBM. This notebook and its source code are released under the terms of the MIT License.